## _*Using Qiskit Aqua algorithms, a how to guide*_

This notebook demonstrates how to use the `Qiskit Aqua` library to invoke an algorithm and process the result.

Further information may be found for the algorithms in the online [Aqua documentation](https://qiskit.org/documentation/aqua/algorithms.html).

Algorithms in Aqua can be created and run as usual in Python by constructing instances and calling methods. There is also a high level `run_algorithm` method that takes a configuration dictionary with data describing which algorithm to use, which components etc along with an InputInstance type to supply data to the algorithm. This latter approach is what we call `declarative` with the former, the regular Python way, `programmatic`. This tutorial will show both approaches.

Aqua has many `algorithms` for solving different problems. For some we also have classical algorithms, that take the exact same input data, to solve the problem. This can be useful in the near term as Quantum algorithms are developed since we are still at a stage where we can do classical comparison of the result.

Aqua also has various `components` which are dependent objects used by algorithms, such as variational forms, qfts, initial states etc. We will see more on this below.

Lastly for developers we also have a collections of `circuits` and gates which can be used to help build out new components and algorithms.

Here we will choose to show some of the main aspects of Aqua by solving a ground state energy problem.

In [13]:
from qiskit.aqua import Operator

As input, for an energy problem, we need a Hamiltonian and so we first create a suitable `Operator ` instance. In this case we have a paulis list, as below, from a previously computed Hamiltonian, that we saved, so as to focus this notebook on using the algorithms. We simply load these paulis to create the original Operator.

This Hamiltonian was created originally using Qiskit Chemistry for an H2 molecule at 0.735A interatomic distance. Please refer to the chemistry tutorials here if you are interested in understanding more. Suffice to say at this level Aqua does not really care about the source of the Operator.

In [14]:
pauli_dict = {
    'paulis': [{"coeff": {"imag": 0.0, "real": -1.052373245772859}, "label": "II"},
              {"coeff": {"imag": 0.0, "real": 0.39793742484318045}, "label": "ZI"},
              {"coeff": {"imag": 0.0, "real": -0.39793742484318045}, "label": "IZ"},
              {"coeff": {"imag": 0.0, "real": -0.01128010425623538}, "label": "ZZ"},
              {"coeff": {"imag": 0.0, "real": 0.18093119978423156}, "label": "XX"}
              ]
}

qubit_op = Operator.load_from_dict(pauli_dict)

### Let's start with a classical algorithm

We can now use the Operator without regard to how it was created. We chose to start this tutorial with a classical algorithm as it involves a little less setting up than the `VQE` quantum algorithm we will use later. Here we will use `ExactEigensolver` to compute the minimum eigenvalue of the Operator (Hamiltonian).

#### First let's show the `programmatic` approach.

We construct an `ExactEigensolver` instance, passing in the Operator, and then call `run()` on in order to compute the result. All Aqua algorithms have the run method (it is defined by a base class which all algorithms extend) and while no parameters are need for classical algorithms a quantum algorithm will require a backend (quantum simulator or real device) on which it will be run. The `result` object returned is a dictionary. While the results fields can be different for algorithms solving different problems, and even within a given problem type there may be algorithm specific data returned, for a given problem the fields core to that problem are common across algorithms in order that different algorithms can be chosen to solve the same problem in a consistent fashion.

In [15]:
from qiskit.aqua.algorithms import ExactEigensolver

ee = ExactEigensolver(qubit_op)
result = ee.run()
print(result['energy'])

-1.857275030202378


#### Now let's show the `declarative` approach. 

Here we need to prepare a configuration dictionary of parameters to define the algorithm. Again we  we will use the ExactEigensolver and need to create an `algorithm` where it is named by `name`. The name comes from a `CONFIGURATION` dictionary in the algorithm and this name is registered to the Aqua discovery framework so we can load the corresponding class and run it during the exceution of `run_algorithm`. `run_algorithm` requires the configuration dictionary and input data passed via an InputInstance class. For an energy problem the data is supplied via an EnergyInput (extends InputInstance), other problem types have their own specific InputInstance. `run_algorithm` returns the same dictionary as above (internally it calls the run() method of the algorithm and passes back the result)

Note: there are other fields such `problem` that could have been added below. This field defaults to `energy`, which is what we want so it has been omitted. Defaults are convenient in the declarative form too as algorithms can define for both their properties as well as defaults for dependent components.

In [17]:
from qiskit.aqua import run_algorithm
from qiskit.aqua.input import EnergyInput

aqua_cfg_dict = {
    'algorithm': {
        'name': 'ExactEigensolver'
    }
}

algo_input = EnergyInput(qubit_op)
result = run_algorithm(aqua_cfg_dict, algo_input)
print(result['energy'])

-1.8572750302023808


### Lets switch now to using a Quantum algorithm.

We will use the Variational Quantum Eigensolver (VQE) to solve the same problem as above. As its name implies its uses a variational approach. An ansatz (a variational form) is supplied and using a quantum/classical hybrid technique the energy resulting from evaluating the Operator with the variational form on a quantum backend is taken down to a minimum using a classical optimizer that varies the parameters of the variational form.

#### Lets do the `declarative` approach first this time

In the description above we talked about `VQE` a `variational form` and an `optimizer`. We can now set this up as a dictionary. While we can omit them from the dictionary, such that defaults are used, here we specify them explicitly so we can set their parameters as we desire.

As this is a quantum algorithm we need to specify a backend. Here we use the `statevector_simpulator` from the `qiskit.BasicAer` provider from `Qiskit Terra`. As this is a variational algorithm going from quantum to classical and looping until it finds a minimum it takes a few seconds. The result here is very close to our classical result above.

In [22]:
aqua_cfg_dict = {
    'algorithm': {
        'name': 'VQE',
        'operator_mode': 'matrix'
    },
    'variational_form': {
        'name': 'RYRZ',
        'depth': 3,
        'entanglement': 'linear'
    },
    'optimizer': {
        'name': 'L_BFGS_B',
        'maxfun': 1000
    },
    'backend': {
        'name': 'statevector_simulator',
        'provider': 'qiskit.BasicAer'
    }
}

algo_input = EnergyInput(qubit_op)
result = run_algorithm(aqua_cfg_dict, algo_input)
print(result['energy'])

-1.8572750302012253


#### And now `programmatic`
 
Here we create the variational form and optimizer and then pass them to VQE along with the Operator. The backend is created and passed to the algorithm so it can be run there.

In [29]:
from qiskit import BasicAer
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.aqua.components.optimizers import L_BFGS_B

var_form = RYRZ(qubit_op.num_qubits, depth=3, entanglement='linear')
optimizer = L_BFGS_B(maxfun=1000)
vqe = VQE(qubit_op, var_form, optimizer)
backend = BasicAer.get_backend('statevector_simulator')
result = vqe.run(backend)
print(result['energy'])

-1.8572750301886618


While a backend can be passed directly to the quantum algorithm run(), internally it will be detected as such and wrapped as a QuantumInstance. However by doing this explicitly yourself, as below, various parameters governing the execution can be set, including in more advanced cases ability to set noise models, coupling maps etc. The following shows the above but using a QuantumInstance and setting up a default transpiler PassManager for circuit processing.

In [31]:
from qiskit.aqua import QuantumInstance
from qiskit.transpiler import PassManager

var_form = RYRZ(qubit_op.num_qubits, depth=3, entanglement='linear')
optimizer = L_BFGS_B(maxfun=1000)
vqe = VQE(qubit_op, var_form, optimizer)
backend = BasicAer.get_backend('statevector_simulator')
qi = QuantumInstance(backend=backend, pass_manager=PassManager())
result = vqe.run(qi)
print(result['energy'])

-1.8572750302012366


### Concluding

This completes an introduction to programming and using Aqua algorithms. There are plenty of other  tutorials showing Aqua being used to solve other problems, including AI, Finance, Optimization and Chemistry. We encourage you to explore these further and see that various capabilities and techniques employed.